# Předpřipravený tagger

In [ ]:
cs = "Proč chceš stát na dešti? Není o co stát."
en = "They refuse to permit us to obtain the refuse permit."

In [ ]:
import nltk

In [ ]:
en_tok = nltk.word_tokenize(en)
en_tok

In [ ]:
nltk.pos_tag(en_tok)

In [ ]:
from corpy.morphodita import Tagger

In [ ]:
czech_pos_tagger = Tagger("/home/lukes/edu/python/czech-morfflex-pdt-161115/czech-morfflex-pdt-161115-pos_only.tagger")

In [ ]:
list(czech_pos_tagger.tag(cs, sents=True))

# Otagovaný korpus

Možnost efektivně procházet (otagovaný) korpus po větách budeme potřebovat při trénování a testování (načti větu, otaguj, porovnej se standardem, načti další...) našich vlastních taggerů.

In [ ]:
nltk.corpus.brown.tagged_words()

In [ ]:
nltk.corpus.brown.tagged_words(tagset="universal")

In [ ]:
!head -n11 /home/lukes/edu/python/pdt3.train.vrt

In [ ]:
from collections import namedtuple

Token = namedtuple("Token", "word pos")

def vert_sents(vert_path):
    with open(vert_path) as file:
        sent = []
        for line in file:
            line = line.strip()
            if "\t" in line:
                word, _, tag = line.split("\t")
                pos = tag[0]
                sent.append(Token(word, pos))
            elif line.startswith("<s "):
                assert not sent
            elif line == "</s>":
                yield sent
                sent = []            

In [ ]:
sent_generator = vert_sents("/home/lukes/edu/python/pdt3.train.vrt")

In [ ]:
next(sent_generator)

# Vlastní tagger

Jak k problému vůbec přistoupit? Na jakém principu by měl tagger fungovat? Představte si, že chcete vymyslet formální postup, podle něhož by bez znalosti daného jazyka mohl značkování provést i člověk, ale místo člověku ho pak povíte počítači. Nesnažte se rovnou vyřešit všechno, zvolte "inženýrský" přístup -- přednostně se snažte identifikovat a řešit části problému, které mají při minimálním vynaloženém úsilí maximální účinek. Účinek v tomto případě měříme úspěšností taggeru -- procentem značek přidělených tak, aby odpovídaly referenčnímu korpusu (zlatému standardu).

# Nejčastější POS

In [ ]:
pos_fdist = nltk.FreqDist(p for s in vert_sents("/home/lukes/edu/python/pdt3.train.vrt") for _, p in s)

In [ ]:
def default_tagger(word):
    return pos_fdist.max()

In [ ]:
default_tagger("kočka")

In [ ]:
default_tagger("sadlfkas")

# Nejčastější POS pro daný tvar

In [ ]:
unigram_cfdist = nltk.ConditionalFreqDist((w, p) for s in vert_sents("/home/lukes/edu/python/pdt3.train.vrt") for w, p in s)

In [ ]:
unigram_cfdist["včera"].max()

In [ ]:
unigram_cfdist["stát"].max()

In [ ]:
unigram_cfdist["asdfsaf"].max()

# Přes regulární výrazy

In [ ]:
regex_tagger = nltk.RegexpTagger([
    (r".*ání$", "N"),
    (r".*[áí]t$", "V"),
    (r"a|i|(proto|tak)?že|když|aby|nebo|ani", "J"),
], backoff=nltk.DefaultTagger("N"))

In [ ]:
regex_tagger.tag(["tání"])

In [ ]:
regex_tagger.tag(["kočka"])

In [ ]:
regex_tagger.evaluate(list(vert_sents("/home/lukes/edu/python/pdt3.test.vrt")))

# N-gramový tagger

![n-gram tagger](http://www.nltk.org/images/tag-context.png)

In [ ]:
bigram_cfdist = nltk.ConditionalFreqDist(((b1.pos, b2.word), b2.pos) for s in vert_sents("/home/lukes/edu/python/pdt3.train.vrt") for b1, b2 in nltk.bigrams(s))

In [ ]:
for k, v in bigram_cfdist.items():
    if len(v) > 1:
        print(k, dict(v))

In [ ]:
t2 = nltk.BigramTagger(vert_sents("/home/lukes/edu/python/pdt3.train.vrt"))
t2.evaluate(list(vert_sents("/home/lukes/edu/python/pdt3.test.vrt")))

In [ ]:
t0 = nltk.DefaultTagger("N")
t1 = nltk.UnigramTagger(vert_sents("/home/lukes/edu/python/pdt3.train.vrt"), backoff=t0)
t2 = nltk.BigramTagger(vert_sents("/home/lukes/edu/python/pdt3.train.vrt"), backoff=t1)
t2.evaluate(list(vert_sents("/home/lukes/edu/python/pdt3.test.vrt")))